In [1]:
!pip install pymorphy3

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import numpy as np

1. Читаем файл, загруженный с сайта http://dict.ruslang.ru/freq.php

In [2]:
df = pd.read_csv("freqrnc2011.csv", sep="\t", encoding="utf-8")
df

,Lemma,PoS,Freq(ipm),R,D,Doc
0,а,conj,8198.0,100,97,32332
1,а,intj,19.8,99,90,757
2,а,part,6.1,59,79,128
3,а,s,2.7,59,85,160
4,аа,intj,1.5,47,80,68
...,...,...,...,...,...,...
52133,ящерица,s,3.6,77,74,158
52134,ящерка,s,0.4,27,82,33
52135,ящик,s,75.4,100,94,1810
52136,ящичек,s,3.4,80,89,212


2. Удаляем ненужные столбцы и приводим все значения в столбце Lemma к нижнем регистру, заменяем ё на е

In [3]:
df = df.drop(['PoS', 'R', 'D', 'Doc'], axis=1)
df["Lemma"] = df["Lemma"].astype(str).str.lower().str.replace("ё","е").str.strip()
df

,Lemma,Freq(ipm)
0,а,8198.0
1,а,19.8
2,а,6.1
3,а,2.7
4,аа,1.5
...,...,...
52133,ящерица,3.6
52134,ящерка,0.4
52135,ящик,75.4
52136,ящичек,3.4


3. Добавляем столбец с абсолютными частотами, зная, что суммарно оценивались 9 200 000 слов (первоначальный столбец частоты удаляем), а также удаляем повторяющиеся строки

In [4]:
df["count"] = (df["Freq(ipm)"].astype(float) * 92000000 / 1000000).round().astype(int)
df = df.drop(['Freq(ipm)'], axis=1)
df = df.groupby("Lemma", as_index=False)[["count"]].sum().sort_values("Lemma").reset_index(drop=True)
df

,Lemma,count
0,а,756847
1,а-а,1638
2,а-а-а,616
3,а-а-а-а,74
4,а-ля,193
...,...,...
51677,ящерица,331
51678,ящерка,37
51679,ящик,6937
51680,ящичек,313


4. Формируем таблицу со всеми вариантами словоформ по всем леммам (каждой словоформе приписывается значение частоты как у леммы, к которой оно относится)

In [5]:
from pymorphy3 import MorphAnalyzer

In [6]:
morph = MorphAnalyzer()

def generate_word_forms_with_count(row):
    lemma = row['Lemma']
    count = row['count']
    forms_data = []
    parsed_word = morph.parse(lemma)[0]
    if parsed_word.lexeme:
        for form in parsed_word.lexeme:
            forms_data.append({'Lemma': lemma, 'word_form': form.word, 'count': count})
    else:
         forms_data.append({'Lemma': lemma, 'word_form': lemma, 'count': count}) # Include lemma itself if no forms generated
    return forms_data

new_rows = df.apply(generate_word_forms_with_count, axis=1).explode()
df_forms = pd.DataFrame([item for item in new_rows])

df_forms

,Lemma,word_form,count
0,а,а,756847
1,а-а,а-а,1638
2,а-а-а,а-а-а,616
3,а-а-а-а,а-а-а-а,74
4,а-ля,а-ля,193
...,...,...,...
1728716,ящур,ящуров,46
1728717,ящур,ящурам,46
1728718,ящур,ящуры,46
1728719,ящур,ящурами,46


5. Удаляем столбец Lemma, и формируем стоимость каждой слово-формы

In [7]:
df_forms = df_forms.drop(["Lemma"], axis=1)

In [8]:
N = df_forms["count"].sum()
df_forms["p"] = (df_forms["count"]) / N
df_forms["cost"] = - np.log(df_forms["p"])
df_forms

,word_form,count,p,cost
0,а,756847,3.659432e-04,7.913032
1,а-а,1638,7.919896e-07,14.048718
2,а-а-а,616,2.978422e-07,15.026702
3,а-а-а-а,74,3.577975e-08,17.145884
4,а-ля,193,9.331745e-08,16.187259
...,...,...,...,...
1728716,ящуров,46,2.224147e-08,17.621307
1728717,ящурам,46,2.224147e-08,17.621307
1728718,ящуры,46,2.224147e-08,17.621307
1728719,ящурами,46,2.224147e-08,17.621307


6. Очищаем таблицу от ненужных столбцом и сохраняем

In [9]:
df_cost = df_forms.drop(["count", "p"], axis=1)
df_cost

,word_form,cost
0,а,7.913032
1,а-а,14.048718
2,а-а-а,15.026702
3,а-а-а-а,17.145884
4,а-ля,16.187259
...,...,...
1728716,ящуров,17.621307
1728717,ящурам,17.621307
1728718,ящуры,17.621307
1728719,ящурами,17.621307


In [10]:
df_cost.to_csv("word_forms_with_cost.csv", index=False)